In [1]:
import subprocess
import json
import os

In [21]:
def measure_test_coverage(file_path):
  try:
    subprocess.run(['cd ' + file_path + ' && npx nyc --exclude examples --exclude test --exclude benchmarks --reporter=json-summary npm test'], shell=True, capture_output=True, text=True, check=True)
    with open(file_path + '/coverage/coverage-summary.json', "r") as coverage_summary: 
      coverage_info = json.load(coverage_summary)
    
    return coverage_info
    
  except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")
    return None  

In [17]:
def compute_cyclomatic_complexity(file_path):
  try:
      result = subprocess.run(['npx cyclomatic-complexity ' + file_path + '\'/**/*.[tj]s\'' + ' -j'], shell=True, capture_output=True, text=True, check=True)
      complexity_data = json.loads(result.stdout)
      
      files = complexity_data
      complexity_info = [{'name': file['file'], 'cyclomatic': file['complexitySum']} for file in files]
      
      return complexity_info
  
  except subprocess.CalledProcessError as e:
      print(f"An error occurred: {e}")
      return None

In [24]:
def main():
  # Measure test coverage before changes
  # Measure complexity of code files
  # Have LLM refactor file
  # see if tests pass
  # Measure test coverage after changes
  # Measure complexity after changes
  # Provide summary of how values have changed

  project_dir = '/media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express'
  code_dir = '/lib'
  
  coverage_info = measure_test_coverage(project_dir)
  complexity_info = compute_cyclomatic_complexity(project_dir + code_dir)

  for module in coverage_info:
      print(f"Module {module} has a branch coverage of {coverage_info[module]['branches']['pct']}")
  
  print()
  for info in sorted(complexity_info, key=lambda info: info['cyclomatic'], reverse=True):
      print(f"Module {info['name']} has a cyclomatic complexity of {info['cyclomatic']}")

if __name__ == "__main__":
    main()


Module total has a branch coverage of 97.05
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/index.js has a branch coverage of 100
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/application.js has a branch coverage of 100
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/express.js has a branch coverage of 100
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/request.js has a branch coverage of 95.31
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/response.js has a branch coverage of 96.44
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/utils.js has a branch coverage of 100
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/view.js has a branch coverage of 95.83
Module /media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express/lib/middleware/init.js has a branch coverage of 